# -------SVM Optimization on UCI Dataset------

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import requests
import io
import time
import random
from tqdm import tqdm

In [12]:
np.random.seed(42)
random.seed(42)

## Function to download and prepare UCI dataset (Iris dataset)

In [13]:
def load_wine_dataset():
    """
    Load the Wine dataset from UCI Machine Learning Repository
    Dataset has 178 instances with 13 features and 3 classes
    """
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"

    # Column names for the wine dataset
    column_names = [
        "Class",
        "Alcohol",
        "Malic acid",
        "Ash",
        "Alcalinity of ash",
        "Magnesium",
        "Total phenols",
        "Flavanoids",
        "Nonflavanoid phenols",
        "Proanthocyanins",
        "Color intensity",
        "Hue",
        "OD280/OD315 of diluted wines",
        "Proline"
    ]

    try:
        # Download the dataset from UCI
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for HTTP errors

        # Read the data into a pandas DataFrame
        data = pd.read_csv(io.StringIO(response.text), header=None, names=column_names)

        print(f"Dataset loaded successfully: {len(data)} rows")

        # Extract features and target
        X = data.iloc[:, 1:].values  # All columns except the first one (class)
        y = data.iloc[:, 0].values   # First column is the class label

        # Basic dataset info
        print(f"Dataset shape: {X.shape}")
        print(f"Number of classes: {len(np.unique(y))}")
        print(f"Class distribution: {np.bincount(y)}")

        return X, y, data
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None, None, None

## Function to create multiple train-test splits

In [14]:
def create_multiple_samples(X, y, n_samples=10, test_size=0.3):
    """
    Create multiple train-test splits for the dataset

    Args:
        X: Feature matrix
        y: Target vector
        n_samples: Number of different splits to create
        test_size: Proportion of the dataset to include in the test split

    Returns:
        List of tuples (X_train, X_test, y_train, y_test)
    """
    samples = []

    for i in range(n_samples):
        # Create a different random split each time
        X_train, X_test, y_train, y_test = train_test_split(
            X, y,
            test_size=test_size,
            random_state=i*10,  # Different seed for each split
            stratify=y  # Maintain class distribution in train and test sets
        )

        samples.append((X_train, X_test, y_train, y_test))

    return samples

## Function to optimize SVM parameters

In [15]:
def optimize_svm(X_train, X_test, y_train, y_test, n_iterations=100):
    """
    Optimize SVM parameters using random search

    Args:
        X_train, X_test, y_train, y_test: Train and test data
        n_iterations: Number of random parameter combinations to try

    Returns:
        Dictionary containing:
        - best_params: The best parameters found
        - best_accuracy: The accuracy achieved with the best parameters
        - history: List of (iteration, params, accuracy) for plotting convergence
    """
    # Scale the features for better SVM performance
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Parameter search space
    kernels = ['linear', 'poly', 'rbf', 'sigmoid']

    # Initialize variables to keep track of the best model
    best_accuracy = 0.0
    best_params = {}
    history = []

    # Random search for optimization
    for i in tqdm(range(n_iterations)):
        # Randomly select parameters
        kernel = random.choice(kernels)

        # Different parameter ranges based on kernel type
        if kernel == 'linear':
            C = 10 ** np.random.uniform(-3, 3)
            params = {'kernel': kernel, 'C': C, 'nu': None, 'epsilon': None}
        else:
            C = 10 ** np.random.uniform(-3, 3)
            gamma = 10 ** np.random.uniform(-4, 0)

            # For polynomial kernel, also select degree
            if kernel == 'poly':
                degree = random.randint(2, 5)
                params = {'kernel': kernel, 'C': C, 'gamma': gamma, 'degree': degree,
                          'nu': None, 'epsilon': None}
            else:
                params = {'kernel': kernel, 'C': C, 'gamma': gamma,
                          'nu': None, 'epsilon': None}

        # Train SVM with the selected parameters
        try:
            svm = SVC(
                kernel=kernel,
                C=C,
                gamma=gamma if 'gamma' in params else 'scale',
                degree=params.get('degree', 3) if kernel == 'poly' else 3,
                random_state=42
            )

            svm.fit(X_train_scaled, y_train)

            # Evaluate the model
            y_pred = svm.predict(X_test_scaled)
            accuracy = accuracy_score(y_test, y_pred)

            # Store history for convergence plot
            history.append((i, params, accuracy))

            # Update best model if current is better
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = params.copy()

        except Exception as e:
            print(f"Error with parameters {params}: {e}")
            continue

    return {
        'best_params': best_params,
        'best_accuracy': best_accuracy,
        'history': history
    }


## Function to plot the convergence graph

In [16]:
def plot_convergence(history, title="SVM Optimization Convergence"):
    """
    Plot the convergence of the optimization process

    Args:
        history: List of (iteration, params, accuracy) from the optimization
        title: Title for the plot
    """
    iterations = [h[0] for h in history]
    accuracies = [h[2] for h in history]

    # Create a list of best accuracies so far at each iteration
    best_so_far = []
    best = 0
    for acc in accuracies:
        if acc > best:
            best = acc
        best_so_far.append(best)

    plt.figure(figsize=(10, 6))
    plt.plot(iterations, best_so_far, 'b-', linewidth=2)
    plt.title(title)
    plt.xlabel('Iteration')
    plt.ylabel('Accuracy (best so far)')
    plt.grid(True)
    plt.ylim(0, 1.05)

    # Save the plot
    plt.savefig('svm_convergence.png', dpi=300, bbox_inches='tight')
    plt.close()

# Main function
def main():
    # Load the Wine dataset
    X, y, data = load_wine_dataset()

    if X is None:
        print("Failed to load dataset. Exiting.")
        return

    # Create 10 different train-test splits
    print("Creating 10 different train-test splits...")
    samples = create_multiple_samples(X, y, n_samples=10, test_size=0.3)

    # Initialize results table
    results = []
    best_sample_idx = 0
    best_overall_accuracy = 0
    best_sample_history = None

    # Process each sample
    for i, (X_train, X_test, y_train, y_test) in enumerate(samples):
        print(f"\nOptimizing SVM for Sample {i+1}/10")

        # Optimize SVM parameters
        optimization_result = optimize_svm(X_train, X_test, y_train, y_test, n_iterations=100)

        # Extract results
        best_params = optimization_result['best_params']
        best_accuracy = optimization_result['best_accuracy']
        history = optimization_result['history']

        # Update best overall sample
        if best_accuracy > best_overall_accuracy:
            best_overall_accuracy = best_accuracy
            best_sample_idx = i
            best_sample_history = history

        # Add to results table
        results.append({
            'Sample': f"S{i+1}",
            'Best Accuracy': best_accuracy,
            'Kernel': best_params['kernel'],
            'C': best_params.get('C'),
            'Gamma': best_params.get('gamma', 'N/A'),
            'Degree': best_params.get('degree', 'N/A') if best_params['kernel'] == 'poly' else 'N/A',
            'Nu': best_params.get('nu', 'N/A'),
            'Epsilon': best_params.get('epsilon', 'N/A')
        })

    # Convert results to DataFrame for better display
    results_df = pd.DataFrame(results)
    print("\nResults Table:")
    print(results_df)

    # Save results to CSV
    results_df.to_csv('svm_optimization_results.csv', index=False)

    # Plot convergence graph for the best sample
    print(f"\nPlotting convergence graph for best sample S{best_sample_idx+1}...")
    plot_convergence(best_sample_history, f"SVM Optimization Convergence for Sample S{best_sample_idx+1}")

    # Basic data analytics of the selected dataset
    print("\nBasic Data Analytics of the Wine Dataset:")
    print(f"Dataset shape: {data.shape}")
    print(f"Number of classes: {len(data['Class'].unique())}")
    print(f"Class distribution: {data['Class'].value_counts()}")

    # Summary statistics
    print("\nSummary Statistics:")
    print(data.describe())

    # Correlation matrix (display top correlations)
    corr_matrix = data.corr()
    print("\nTop Feature Correlations with Class:")
    print(corr_matrix['Class'].sort_values(ascending=False))

    # Generate correlation heatmap
    plt.figure(figsize=(12, 10))
    plt.matshow(corr_matrix)
    plt.title('Feature Correlation Matrix')
    plt.colorbar()
    plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
    plt.close()

    print("\nAnalysis complete! Results saved to CSV and plots saved as PNG files.")
    print("You can now upload these to GitHub along with the code.")

if __name__ == "__main__":
    start_time = time.time()
    main()
    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Dataset loaded successfully: 178 rows
Dataset shape: (178, 13)
Number of classes: 3
Class distribution: [ 0 59 71 48]
Creating 10 different train-test splits...

Optimizing SVM for Sample 1/10


100%|██████████| 100/100 [00:00<00:00, 338.43it/s]



Optimizing SVM for Sample 2/10


100%|██████████| 100/100 [00:00<00:00, 325.73it/s]



Optimizing SVM for Sample 3/10


...

100%|██████████| 100/100 [00:00<00:00, 334.30it/s]



Optimizing SVM for Sample 10/10


100%|██████████| 100/100 [00:00<00:00, 346.89it/s]



Results Table:
  Sample  Best Accuracy   Kernel           C     Gamma Degree    Nu Epsilon
0     S1       1.000000   linear    0.176702       N/A    N/A  None    None
1     S2       1.000000  sigmoid    0.124525  0.080135    N/A  None    None
2     S3       0.981481   linear    0.049227       N/A    N/A  None    None
3     S4       1.000000      rbf  394.794324  0.291243    N/A  None    None
4     S5       1.000000  sigmoid   53.899318  0.000404    N/A  None    None
5     S6       1.000000  sigmoid  238.789327  0.000113    N/A  None    None
6     S7       1.000000     poly    0.002224  0.752334      3  None    None
7     S8       1.000000      rbf    3.103333  0.001191    N/A  None    None
8     S9       1.000000      rbf  617.606956  0.000105    N/A  None    None
9    S10       1.000000   linear    0.022905       N/A    N/A  None    None

Plotting convergence graph for best sample S1...

Basic Data Analytics of the Wine Dataset:
Dataset shape: (178, 14)
Number of classes: 3
Class dis

<Figure size 1200x1000 with 0 Axes>

# -------Wine Dataset Exploratory Analysis-------

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import requests
import io
import seaborn as sns

## # Function to download and prepare UCI dataset

In [10]:
def load_wine_dataset():
    """
    Load the Wine dataset from UCI Machine Learning Repository and perform
    exploratory data analysis
    """
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"

    # Column names for the wine dataset
    column_names = [
        "Class",
        "Alcohol",
        "Malic acid",
        "Ash",
        "Alcalinity of ash",
        "Magnesium",
        "Total phenols",
        "Flavanoids",
        "Nonflavanoid phenols",
        "Proanthocyanins",
        "Color intensity",
        "Hue",
        "OD280/OD315 of diluted wines",
        "Proline"
    ]

    try:
        # Download the dataset from UCI
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for HTTP errors

        # Read the data into a pandas DataFrame
        data = pd.read_csv(io.StringIO(response.text), header=None, names=column_names)

        print(f"Dataset loaded successfully: {len(data)} rows")
        return data

    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

def analyze_dataset(data):
    """
    Perform detailed exploratory data analysis on the dataset
    """
    if data is None:
        print("No data to analyze.")
        return

    # Basic dataset information
    print("\n=== DATASET OVERVIEW ===")
    print(f"Shape: {data.shape}")
    print(f"Number of classes: {len(data['Class'].unique())}")
    print(f"Class distribution: {data['Class'].value_counts().to_dict()}")

    # Summary statistics
    print("\n=== SUMMARY STATISTICS ===")
    print(data.describe())

    # Check for missing values
    print("\n=== MISSING VALUES ===")
    missing = data.isnull().sum()
    print(missing if missing.sum() > 0 else "No missing values found.")

    # Feature distributions by class
    print("\n=== FEATURE ANALYSIS ===")

    # Prepare data for scaling
    X = data.iloc[:, 1:].values
    y = data.iloc[:, 0].values

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    data_scaled = pd.DataFrame(X_scaled, columns=data.columns[1:])
    data_scaled['Class'] = y

    # Feature importance - correlation with class
    corr_with_class = data.corr()['Class'].sort_values(ascending=False)
    print("\nFeature Correlation with Class:")
    print(corr_with_class)

    # Generate visualizations
    create_visualizations(data, data_scaled)

    return data

def create_visualizations(data, data_scaled):
    """
    Create and save visualizations for the dataset
    """
    # Set aesthetics for plots
    sns.set(style="whitegrid")

    # 1. Class distribution
    plt.figure(figsize=(8, 6))
    sns.countplot(x='Class', data=data)
    plt.title('Distribution of Wine Classes')
    plt.xlabel('Wine Class')
    plt.ylabel('Count')
    plt.savefig('class_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 2. Correlation heatmap
    plt.figure(figsize=(14, 12))
    corr_matrix = data.corr()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', mask=mask,
                linewidths=0.5, cbar_kws={"shrink": .8})
    plt.title('Feature Correlation Matrix')
    plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 3. Feature distributions by class - Select top 6 correlated features
    top_features = data.corr()['Class'].abs().sort_values(ascending=False)[1:7].index

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()

    for i, feature in enumerate(top_features):
        sns.boxplot(x='Class', y=feature, data=data, ax=axes[i])
        axes[i].set_title(f'{feature} by Wine Class')
        axes[i].set_xlabel('Wine Class')
        axes[i].set_ylabel(feature)

    plt.tight_layout()
    plt.savefig('feature_distributions.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. Pairplot of most important features
    plt.figure(figsize=(14, 10))
    sns.pairplot(data, vars=top_features[:4], hue='Class', palette='viridis')
    plt.savefig('feature_pairplot.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 5. Feature importance based on correlation
    plt.figure(figsize=(10, 8))
    feature_importance = data.corr()['Class'].abs().sort_values(ascending=False)[1:]
    sns.barplot(x=feature_importance.values, y=feature_importance.index)
    plt.title('Feature Importance Based on Correlation with Class')
    plt.xlabel('Absolute Correlation Coefficient')
    plt.tight_layout()
    plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
    plt.close()

    print("Visualizations saved to current directory.")

if __name__ == "__main__":
    print("Running Wine Dataset Exploratory Analysis...")
    wine_data = load_wine_dataset()
    analyze_dataset(wine_data)
    print("\nAnalysis complete!")

Running Wine Dataset Exploratory Analysis...
Dataset loaded successfully: 178 rows

=== DATASET OVERVIEW ===
Shape: (178, 14)
Number of classes: 3
Class distribution: {2: 71, 1: 59, 3: 48}

=== SUMMARY STATISTICS ===
            Class     Alcohol  Malic acid         Ash  Alcalinity of ash  \
count  178.000000  178.000000  178.000000  178.000000         178.000000   
mean     1.938202   13.000618    2.336348    2.366517          19.494944   
std      0.775035    0.811827    1.117146    0.274344           3.339564   
min      1.000000   11.030000    0.740000    1.360000          10.600000   
25%      1.000000   12.362500    1.602500    2.210000          17.200000   
50%      2.000000   13.050000    1.865000    2.360000          19.500000   
75%      3.000000   13.677500    3.082500    2.557500          21.500000   
max      3.000000   14.830000    5.800000    3.230000          30.000000   

        Magnesium  Total phenols  Flavanoids  Nonflavanoid phenols  \
count  178.000000     178.00

<Figure size 1400x1000 with 0 Axes>